<a href="https://colab.research.google.com/github/Danish-Hussain-DSAI/NLP-project-NUST-/blob/main/NLP_chatbot_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path="/content/drive/MyDrive/NKP project/topical_chat.csv"

In [ ]:
import pandas as pd
df = pd.read_csv(data_path)
df

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper
...,...,...,...
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper
188375,8628,I don't think I have either. That's an insane...,Curious to dive deeper
188376,8628,"I did, my little brother used to love Thomas ...",Happy


In [ ]:
df.columns

Index(['conversation_id', 'message', 'sentiment'], dtype='object')

In [ ]:
import re

# Remove any rows with missing values
df.dropna(inplace=True)


In [ ]:
# Strip leading and trailing whitespace from the text
df["message"] = df["message"].str.strip()

In [ ]:
# Drop rows with empty text
df = df[df["message"] != ""]

In [ ]:
# Define a function to normalize text
def normalize_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^a-z0-9\s]", "", text)  # Remove punctuation and special characters
    text = re.sub(r"\s+", " ", text)  # Remove extra spaces
    return text

In [ ]:
# Apply the normalize_text function to the 'text' column
df["message"] = df["message"].apply(normalize_text)

<ipython-input-8-b2f350c2f00f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["message"] = df["message"].apply(normalize_text)


In [ ]:
# Reset the index after cleaning and normalization
df.reset_index(drop=True, inplace=True)

In [ ]:
# Print the normalized DataFrame
print(df)

        conversation_id                                            message  \
0                     1               are you a fan of google or microsoft   
1                     1  both are excellent technology they are helpful...   
2                     1  im not a huge fan of google but i use it a lot...   
3                     1  google provides online related services and pr...   
4                     1  yeah their services are good im just not a fan...   
...                 ...                                                ...   
188368             8628  wow it does not seem like that long since i me...   
188369             8628  i havent seen that episode i might google it l...   
188370             8628  i dont think i have either thats an insane amo...   
188371             8628  i did my little brother used to love thomas th...   
188372             8628  it did ringo starr george carlin and alec bald...   

                      sentiment  
0        Curious to dive deep

In [ ]:
# Remove the last row
df = df.drop(df.index[-1])
df=df[["message"]]

In [ ]:
df

,message
0,are you a fan of google or microsoft
1,both are excellent technology they are helpful...
2,im not a huge fan of google but i use it a lot...
3,google provides online related services and pr...
4,yeah their services are good im just not a fan...
...,...
188367,its like 750 pokemon episodes and its been aro...
188368,wow it does not seem like that long since i me...
188369,i havent seen that episode i might google it l...
188370,i dont think i have either thats an insane amo...


In [ ]:
# Separate questions and answers into X and Y lists
x = df[df.index % 2 == 0]["message"].tolist()  # Questions
y = df[df.index % 2 == 1]["message"].tolist()  # Answers

In [ ]:
print(x[0])
print(y[0])

are you a fan of google or microsoft
both are excellent technology they are helpful in many ways for the security purpose both are super


In [ ]:
print(len(x))
len((y))

94186


94186

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Set a padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [ ]:
# Tokenize the questions and answers using the GPT-2 tokenizer
tokenized_X = tokenizer(x, return_tensors="pt", padding=True, truncation=True)
tokenized_Y = tokenizer(y, return_tensors="pt", padding=True, truncation=True)

In [ ]:
# Prepare a dictionary with tokens and their corresponding IDs
token_dict = {
    "input_ids_X": tokenized_X["input_ids"],
    "attention_mask_X": tokenized_X["attention_mask"],
    "input_ids_Y": tokenized_Y["input_ids"],
    "attention_mask_Y": tokenized_Y["attention_mask"],
}

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(
    token_dict["input_ids_X"],
    token_dict["input_ids_Y"],
    test_size=0.2,  # Adjust the test size as needed
    random_state=42  # Set a random seed for reproducibility
)

In [ ]:
!pip install accelerate

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import Trainer, TrainingArguments

# Initialize the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # You can choose other GPT-2 variants like "gpt2-medium" or "gpt2-large"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Convert tokenized data to PyTorch tensors
X_train_tensors = torch.tensor(X_train)
X_test_tensors = torch.tensor(X_test)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./chatbot_model",  # Directory to save the model
    per_device_train_batch_size=4,  # Batch size for training
    per_device_eval_batch_size=4,   # Batch size for evaluation
    num_train_epochs=3,             # Number of training epochs
    evaluation_strategy="steps",    # Evaluate at the end of each epoch
    save_total_limit=1,             # Number of saved checkpoints
    save_steps=1000,                # Save a checkpoint every 1000 steps
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=X_train_tensors,  # Assuming your data is in X_train_tensors
    eval_dataset=X_test_tensors,    # Assuming your data is in X_test_tensors
)

# Train the model
trainer.train()

# Evaluate the model on the test data
results = trainer.evaluate()

# Print evaluation results (you can customize this)
for key, value in results.items():
    print(f"{key}: {value}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 2.0 MB/s eta 0:00:00


<ipython-input-34-085432e3004e>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensors = torch.tensor(X_train)
<ipython-input-34-085432e3004e>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensors = torch.tensor(X_test)


ImportError: ignored